In [6]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [7]:
df1=pd.read_csv("2006.csv")
df2=pd.read_csv("2007.csv")
df3=pd.read_csv("2008.csv")
df4=pd.read_csv("2009.csv")
df5=pd.read_csv("2010.csv")

In [8]:
df=pd.concat([df1,df2,df3,df4,df5])

In [9]:
df

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90
...,...,...,...,...,...
9563,15.12,48.92,1011.80,72.93,462.59
9564,33.41,77.95,1010.30,59.72,432.90
9565,15.99,43.34,1014.20,78.66,465.96
9566,17.65,59.87,1018.58,94.65,450.93


In [16]:
train, test = train_test_split(df, test_size=0.25, random_state=20)
train, val = train_test_split(train, test_size=0.25, random_state=20)


scaler_x = StandardScaler()
scaler_y = StandardScaler()

train_x = scaler_x.fit_transform(train.iloc[:,:-1].values)
train_y = scaler_y.fit_transform(train.iloc[:,-1].values.reshape(-1,1))

val_x = scaler_x.transform(val.iloc[:,:-1].values)
val_y = scaler_y.transform(val.iloc[:,-1].values.reshape(-1,1))

test_x = scaler_x.transform(test.iloc[:,:-1].values)
test_y = scaler_y.transform(test.iloc[:,-1].values.reshape(-1,1))

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(4,36)
        self.fc2 = nn.Linear(36,69)
        self.fc3 = nn.Linear(69,10)
        self.fc4 = nn.Linear(10,1)
    def forward(self,x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x=self.fc4(x)
        return x

In [18]:
net = Net()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [19]:
for epoch in range(690):
    optimizer.zero_grad()
    outputs = net(torch.Tensor(train_x))
    loss = criterion(outputs, torch.Tensor(train_y))
    loss.backward()
    optimizer.step()

    if epoch % 69 == 0:
        val_outputs = net(torch.Tensor(val_x))
        val_loss = criterion(val_outputs, torch.Tensor(val_y))
        print("Epoch: {}, Loss: {}, Validation Loss: {}".format(epoch, loss.item(), val_loss.item()))


Epoch: 0, Loss: 1.043221354484558, Validation Loss: 1.04315984249115
Epoch: 69, Loss: 0.09028919041156769, Validation Loss: 0.08693946897983551
Epoch: 138, Loss: 0.0631645992398262, Validation Loss: 0.06128618121147156
Epoch: 207, Loss: 0.0603475421667099, Validation Loss: 0.05850203335285187
Epoch: 276, Loss: 0.05928535759449005, Validation Loss: 0.05747358128428459
Epoch: 345, Loss: 0.05839649215340614, Validation Loss: 0.0565689317882061
Epoch: 414, Loss: 0.057377900928258896, Validation Loss: 0.05554617568850517
Epoch: 483, Loss: 0.05630778893828392, Validation Loss: 0.05445227771997452
Epoch: 552, Loss: 0.05531662330031395, Validation Loss: 0.05346648395061493
Epoch: 621, Loss: 0.05437080189585686, Validation Loss: 0.052460458129644394


In [20]:
test_outputs = net(torch.Tensor(test_x))
print(test_outputs)

tensor([[-1.1508],
        [-0.4280],
        [-0.4246],
        ...,
        [-0.1190],
        [-0.0201],
        [ 1.0865]], grad_fn=<AddmmBackward0>)


In [21]:
test_loss = criterion(test_outputs,torch.Tensor(test_y))
print(f"Test loss: {test_loss.item()}")

Test loss: 0.052799053490161896
